# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [82]:
# import libraries
import pandas as pd
import numpy as np
import re
import nltk
nltk.download(['punkt', 'wordnet'])

from nltk.stem import WordNetLemmatizer
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, f1_score,\
precision_score, recall_score, make_scorer
import pickle

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [83]:
# load data from database
engine = create_engine('sqlite:///DisasterCategories.db')
df = pd.read_sql_table('DisasterCategories',engine)
X = df.message.values
y = df.drop(columns=['id', 'message', 'original', 'genre']).values
category_names = np.array(df.drop(columns=['id', 'message', 'original', 'genre']).columns)

In [84]:
print(category_names)
print(X[:5])


['related' 'request' 'offer' 'aid_related' 'medical_help'
 'medical_products' 'search_and_rescue' 'security' 'military' 'child_alone'
 'water' 'food' 'shelter' 'clothing' 'money' 'missing_people' 'refugees'
 'death' 'other_aid' 'infrastructure_related' 'transport' 'buildings'
 'electricity' 'tools' 'hospitals' 'shops' 'aid_centers'
 'other_infrastructure' 'weather_related' 'floods' 'storm' 'fire'
 'earthquake' 'cold' 'other_weather' 'direct_report']
['Weather update - a cold front from Cuba that could pass over Haiti'
 'Is the Hurricane over or is it not over'
 'Looking for someone but no name'
 'UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.'
 'says: west side of Haiti, rest of the country today and tonight']


### 2. Write a tokenization function to process your text data

In [85]:
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

In [86]:
def tokenize(text):
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens
    pass

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [87]:
pipeline =Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(estimator = RandomForestClassifier(n_jobs=-1)))
    ])


### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [100]:
X_train, X_test, y_train, y_test = train_test_split(X, y)
pipeline.fit(X_train, y_train)


Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [112]:
for i,col in enumerate(category_names):
        print('------------------------')
        print(col)
        
        
        print(classification_report(y_test[i], y_pred[i]))
        print("------------------------------------------------------\n")
      

------------------------
related
             precision    recall  f1-score   support

          0       0.89      1.00      0.94        31
          1       1.00      0.20      0.33         5

avg / total       0.90      0.89      0.86        36

------------------------------------------------------

------------------------
request
             precision    recall  f1-score   support

          0       1.00      0.94      0.97        35
          1       0.33      1.00      0.50         1

avg / total       0.98      0.94      0.96        36

------------------------------------------------------

------------------------
offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00        35
          1       1.00      1.00      1.00         1

avg / total       1.00      1.00      1.00        36

------------------------------------------------------

------------------------
aid_related
             precision    recall  f1-score   support


/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
pipeline.get_params()
parameters = {
        'clf__estimator__min_samples_split':[10,110],
        'clf__estimator__max_depth':[None,20,50],
        'clf__estimator__n_estimators': [100, 250]
}

cv = GridSearchCV(pipeline, param_grid=parameters)
cv.fit(X_train, y_train)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
cv.best_score_
    

In [ ]:
cv.best_params_

In [ ]:
refined_pipeline = cv.best_estimator_

In [ ]:
y_pred_best = refined_pipeline.predict(X_test)

In [ ]:
for i,col in enumerate(category_names):
        print('------------------------')
        print(col)
        
        
        print(classification_report(y_test[i], y_pred_best[i]))
        print("------------------------------------------------------\n")

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
refined_scores = tabulate_metric_scores(y_test, y_pred_best).mean()

refined_scores

In [ ]:
non_refined_scores

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.